<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Задание-1" data-toc-modified-id="Задание-1-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Задание 1</a></span></li><li><span><a href="#Задание-2" data-toc-modified-id="Задание-2-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Задание 2</a></span></li><li><span><a href="#Задание-3" data-toc-modified-id="Задание-3-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Задание 3</a></span></li><li><span><a href="#Задание-4" data-toc-modified-id="Задание-4-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Задание 4</a></span></li><li><span><a href="#Задание-5" data-toc-modified-id="Задание-5-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Задание 5</a></span></li></ul></div>

# Cервис для чтения книг по подписке

**Описание**

Компания владеет крупным сервисом для чтения книг по подписке.<br>
Имеем информацию о книгах, издательствах, авторах, а также пользовательские обзоры книг.<br>
*Таблицы:*
- `books` - данные о книгах
- `authors` - данные об авторах
- `publishers` - данные об издательствах
- `ratings` - данные о пользовательских оценках книг
- `reviews` - данные о пользовательских обзорах на книги


In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# Здесь в виде словаря db_config были прописаны параметры подключения, 
# которые не могут быть отображены в открытом доступе
# имя пользователя, пароль, host, порт подключения, название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [3]:
# функция для выполнения запроса и возвращению результата
def query_result(query):
    return pd.io.sql.read_sql(query, con = engine)

In [4]:
# выведем первые строки в таблицах
query_1 = ''' SELECT * 
                FROM books
               LIMIT 2
          '''
query_2 = ''' SELECT * 
                FROM authors
               LIMIT 2
          '''
query_3 = ''' SELECT * 
                FROM publishers
               LIMIT 2
          '''
query_4 = ''' SELECT * 
                FROM ratings
               LIMIT 2
          '''
query_5 = ''' SELECT * 
                FROM reviews
               LIMIT 2
          '''

for x in [query_1, query_2, query_3, query_4, query_5]:
    display(query_result(x))

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs


,publisher_id,publisher
0,1,Ace
1,2,Ace Book


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...


### Задание 1

Cколько книг вышло после 1 января 2000 года

In [6]:
# DATE_TRUNC используем для сравнения даты с желаемой
query_1 = ''' SELECT COUNT(book_id)
                FROM books
               WHERE DATE_TRUNC('day', publication_date) > '2000-01-01';
          '''
query_result(query_1)

,count
0,819


После 1 января 200 года вышло 819 книг

### Задание 2

Количество обзоров и средняя оценка для каждой книги 

In [7]:
# С помощью подзапросов в разделе WITH посчитаем отдельно количестов обзоров для каждой книги и среднюю оценку
# Объединим эти таблицы по столбцу title и выведем интересующие нас показатели
# среднюю оценку округляем для удобства до 2 знаков - ROUND(AVG(rating),2)
query_2 = ''' WITH
              title_count AS (
                  SELECT b.book_id,
                         b.title,
                         COUNT(review_id)
                    FROM reviews AS re
                    LEFT JOIN books AS b ON re.book_id = b.book_id
                   GROUP BY b.book_id
              ),
              title_avg AS (
                  SELECT b.book_id,
                         b.title,
                         ROUND(AVG(ra.rating),2)
                    FROM ratings AS ra
                    LEFT JOIN books AS b ON ra.book_id = b.book_id
                   GROUP BY b.book_id
              )
              
              SELECT t_c.title,
                     t_c.count,
                     t_a.round
                FROM title_avg AS t_a
                LEFT JOIN title_count AS t_c ON t_a.book_id = t_c.book_id;
          '''
query_result(query_2)

,title,count,round
0,'Salem's Lot,2.0,3.67
1,1 000 Places to See Before You Die,1.0,2.50
2,13 Little Blue Envelopes (Little Blue Envelope...,3.0,4.67
3,1491: New Revelations of the Americas Before C...,2.0,4.50
4,1776,4.0,4.00
...,...,...,...
995,Wyrd Sisters (Discworld #6; Witches #2),3.0,3.67
996,Xenocide (Ender's Saga #3),3.0,3.40
997,Year of Wonders,4.0,3.20
998,You Suck (A Love Story #2),2.0,4.50


Вывели все 1000 книг количеством обзоров и средней оценкой. Для 6 книг отсутвуют обзоры, у таких книг отсутствует количество обзоров, а есть только оценка.

In [8]:
# проверка подзапроса с количеством обзоров
query_2_1 = '''SELECT b.book_id,
                      b.title,
                      COUNT(review_id)
                 FROM reviews AS re
                 LEFT JOIN books AS b ON re.book_id = b.book_id
                GROUP BY b.book_id;
          '''
query_result(query_2_1)

,book_id,title,count
0,652,The Body in the Library (Miss Marple #3),2
1,273,Galápagos,2
2,51,A Tree Grows in Brooklyn,5
3,951,Undaunted Courage: The Pioneering First Missio...,2
4,839,The Prophet,4
...,...,...,...
989,64,Alice in Wonderland,4
990,55,A Woman of Substance (Emma Harte Saga #1),2
991,148,Christine,3
992,790,The Magicians' Guild (Black Magician Trilogy #1),2


In [9]:
# вывод всех книг для которых есть обзоры и количество этих обзоров
query_2_2 = '''SELECT COUNT(review_id)
               FROM reviews
               GROUP BY book_id;
            '''
query_result(query_2_2)

,count
0,2
1,2
2,5
3,2
4,4
...,...
989,4
990,2
991,3
992,2


In [10]:
# проверка подзапроса со средней оценкой
query_2_3 = ''' SELECT b.book_id,
                       b.title,
                       ROUND(AVG(ra.rating),2)
                  FROM ratings AS ra
                  LEFT JOIN books AS b ON ra.book_id = b.book_id
                 GROUP BY b.book_id;
          '''
query_result(query_2_3)

,book_id,title,round
0,652,The Body in the Library (Miss Marple #3),4.50
1,273,Galápagos,4.50
2,51,A Tree Grows in Brooklyn,4.25
3,951,Undaunted Courage: The Pioneering First Missio...,4.00
4,839,The Prophet,4.29
...,...,...,...
995,64,Alice in Wonderland,4.23
996,55,A Woman of Substance (Emma Harte Saga #1),5.00
997,148,Christine,3.43
998,790,The Magicians' Guild (Black Magician Trilogy #1),3.50


### Задание 3

Определение издательства, которое выпустило наибольшее число книг толще 50 страниц

In [11]:
# Присоеденим к таблице с книгами таблицу с издательствами
# b.num_pages > 50 - исключаем книги меньше чем с 50 страницами
# Группируем таблицу по издательствам и считаем для каждого количество книг
# Сортируем по убыванию количества книг и оставляем только верхнее значение(LIMIT 1)
query_3 = ''' SELECT p.publisher,
                     COUNT(b.book_id)
                FROM books AS b
                LEFT JOIN publishers AS p ON b.publisher_id = p.publisher_id
               WHERE b.num_pages > 50
               GROUP BY p.publisher
               ORDER BY count DESC
               LIMIT 1;
          '''
query_result(query_3)

,publisher,count
0,Penguin Books,42


Таким издательством является Penguin Books, оно выпустило 42 книги, толще 50 страниц.

### Задание 4

Определение автора с самой высокой средней оценкой книг — учитываются только книги с 50 и более оценками

In [12]:
# Объединяем таблицы с авторами, книгами и рейтингами
# отфильтруем книги используя подзапрос с 50 и более оценками
# сортируем и оставляем верхнее значение
query_4_1 = ''' SELECT a.author,
                       ROUND(AVG(ra.rating),2)
                  FROM authors AS a
                  LEFT JOIN books AS b ON a.author_id = b.author_id
                  LEFT JOIN ratings AS ra ON b.book_id = ra.book_id
                 WHERE b.book_id IN (SELECT book_id
                                       FROM ratings
                                      GROUP BY book_id
                                     HAVING COUNT(rating_id)>50)
                 GROUP BY a.author
                 ORDER BY round DESC
                 LIMIT 1;
            '''
query_result(query_4_1)

,author,round
0,J.K. Rowling/Mary GrandPré,4.29


In [13]:
# проверка подзапроса
query_4_2 = ''' SELECT book_id
                  FROM ratings
                 GROUP BY book_id
                HAVING COUNT(rating_id) > 50;
          '''
query_result(query_4_2)

,book_id
0,75
1,750
2,545
3,948
4,488
5,696
6,722
7,627
8,733
9,779


У нас есть два автора с одинаковой средней оценкой - это J.K. Rowling и Mary GrandPré

### Задание 5

Подсчет среднего количества обзоров от пользователей, которые поставили больше 50 оценок.

In [14]:
# сначала создадим список из пользователей, которые поставили больше 50 оценок - это будет первый подзапрос
# далее для этих пользователей посчитаем количество сделанных обзоров - это будет второй подзапрос
# теперь осталось просто посчитать среднее значение количества обзоров
query_5 = ''' SELECT ROUND(AVG(count),2)
                FROM (SELECT re.username,
                             COUNT(re.review_id)
                        FROM reviews AS re
                       WHERE re.username IN (SELECT ra.username
                                               FROM ratings AS ra
                                              GROUP BY ra.username
                                             HAVING COUNT(ra.rating_id)>50)
                       GROUP BY re.username) AS user_count;
          '''
query_result(query_5)

,round
0,24.33


Пользователи, поставившие больше 50 оценок, в среднем делают порядка 24 обзоров на книги.

In [15]:
# результаты первого подзапроса
query_5_1 = 
''' 
    SELECT ra.username
      FROM ratings AS ra
     GROUP BY ra.username
    HAVING COUNT(ra.rating_id) > 50;
'''
query_result(query_5_1)

,username
0,sfitzgerald
1,jennifermiller
2,xdavis
3,paul88
4,martinadam
5,richard89


In [16]:
# результаты второго подзапроса
query_5_2 = ''' SELECT re.username,
                       COUNT(re.review_id)
                  FROM reviews AS re
                 WHERE re.username IN (SELECT ra.username
                                         FROM ratings AS ra
                                        GROUP BY ra.username
                                       HAVING COUNT(ra.rating_id)>50)
                 GROUP BY re.username;
          '''
query_result(query_5_2)

,username,count
0,sfitzgerald,28
1,jennifermiller,25
2,xdavis,18
3,paul88,22
4,martinadam,27
5,richard89,26
